In [20]:
from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.adaccount import AdAccount
import pandas as pd
import gspread
from sqlalchemy import create_engine


from gspread_dataframe import set_with_dataframe
from pipedrive.client import Client
from main.apps.pat_bot.models import ZcomProspects
gc = gspread.service_account(filename="./credentials.json")

ModuleNotFoundError: No module named 'main'

In [2]:
import environ
env = environ.Env()
database_url = env.str(
    var="DATABASE_URL", default="postgres://postgres:@postgres:5432/postgres"
)
engine = create_engine(database_url, echo=False)
client = Client(domain='https://companydomain.pipedrive.com/')
client.set_api_token('c70f38803bb436089bce792681cd178e8459c86e')

In [3]:
url_1 = 'https://docs.google.com/spreadsheets/d/1ab4eW4deYLV-3mMgAMuafoWM3e7JjYzoLDAJ83WPuaA'
workbook = gc.open_by_url(url_1)

In [4]:
sheet_1 = workbook.get_worksheet(6)

data_1 = sheet_1.get_all_values()

df_1 = pd.DataFrame(data_1)
df_1.columns = df_1.iloc[0]
df_1 = df_1[1:]
col_name = df_1.columns.to_list()
df_1 = df_1.drop(columns=col_name[16:])


In [5]:
df_1

,Date,Day,Time,Source,Ads id,Campaign name,Ads set name,First Name - Last Name,Company Name,Email,Phone Number,Contact point,Duplicate,Duplicate,Owner,Status
1,Jan 20,Wed,,,,,,,Sport for life,,,Form,Yes,Yes,CDM,Lose
2,Jan 21,Thur,,,,,,,Thaniya Group,,,Call,No,No,Zcom,
3,Feb 5,Fri,,,,,,Parinya Khongpratet,"Mogen (Thailand) Co., Ltd.",parinya@mogen.co.th,06 3216 7978 / 08 6799 2055,Mail,Yes,Yes,CDM,Meeting
4,Feb 9,Tue,,,,,,คุณวราภรณ์,"Osstem (Thailand) Co.,Ltd",,089-693-6868,Call,No,No,Zcom,Call(filter)
5,Feb 13,Sat,10:53 AM,,,,,พศิน ทองเกื้อ,,phasin80@gmail.com,0992214269,,No,No,Zcom,Call(filter)
6,Feb 13,Sat,11:11 AM,,,,,นายชาญชัย คำพันชนะ,สำนักงานอัยการจังหวัดเดชอุดม,chanturbo@gmail.com,0804655782,,No,No,,Meeting
7,Feb 13,Sat,8:25 PM,,,,,Cherdpong Chiwpreecha,สำนักงานตำรวจแห่งชาติ,Interpol6348@gmail.com,0869036666,,Yes,Yes,Zcom,Call(filter)
8,Feb 15,Mon,8:26 AM,,,,,อนุรักษ์ ศรีวรรณ,Konicaminolta,Anurak.Seesuwan@konicaminolta.com,0632096685,,No,No,Zcom,Call(filter)
9,Feb 15,Mon,,Facebook ads,23846787272960107,PRO:EMEM_OBJ:CONV_CN:NORM_AW:SING_CONT:4STEPS,BISS&IT-TECH-150221,คุณกานต์,the prodigy group,thanaphat.in@theprodigygeneration.com,062-396-8289,,Yes,Yes,CDM,Call(sale)


In [6]:
sheet_2 = workbook.get_worksheet(7)
data_2 = sheet_2.get_all_values()

df_2 = pd.DataFrame(data_2)
df_2.columns = df_2.iloc[0]
df_2 = df_2[1:]
df_2['Status']

1             Lose
2                 
3          Meeting
4     Call(filter)
5     Call(filter)
6          Meeting
7     Call(filter)
8     Call(filter)
9       Call(sale)
10                
Name: Status, dtype: object

In [7]:
# eq = [df_1.iloc[i] for i in range(len(df_1)) if df_1['Status'].iloc[i] != df_2['Status'].iloc[i]]

In [8]:
new_df_2 = df_2[len(df_1):]
df_drop_blank = new_df_2.drop(
    new_df_2[(new_df_2.Status) != ""].index
)

In [9]:
df_drop_change = df_2.drop(
    df_1[(df_1.Status) == (df_2[:len(df_1)].Status)].index
)
df_drop_change_fil = df_drop_change.drop(
     df_drop_change[df_drop_change.Status != "Call(filter)"].index
)

In [1]:
def marketing_zcom():
    url = 'https://docs.google.com/spreadsheets/d/1ab4eW4deYLV-3mMgAMuafoWM3e7JjYzoLDAJ83WPuaA'
    workbook = gc.open_by_url(url)
    sheet_1 = workbook.worksheet("Digital MKT lead")

    data_1 = sheet_1.get_all_values()

    df_1 = pd.DataFrame(data_1)
    df_1.columns = df_1.iloc[0]
    df_1 = df_1[1:]
    col_name = df_1.columns.to_list()
    df_1 = df_1.drop(columns=col_name[16:])

    check = engine.has_table("marketing_zcom")

    if not check:
        df_1.to_sql("marketing_zcom", engine, if_exists="replace", index=False)

    data = pd.read_sql("marketing_zcom", engine)
    data = pd.DataFrame(data)
    data.index += 1

    new_data_df = df_1[len(data):]
    df_drop_blank = new_data_df.drop(
        new_data_df[new_data_df.Status != ""].index
    )
    if len(df_drop_blank) > 0:
        print('send P pat')
    
    old_data_df = df_1[:len(data)]
    df_drop_change = old_data_df.drop(
        old_data_df[old_data_df.Status == data.Status].index
    )
    if len(df_drop_change) > 0:
        df_drop_change_fil = df_drop_change.drop(
            df_drop_change[df_drop_change.Status != "Call(filter)"].index
        )
        df_drop_change_sale = df_drop_change.drop(
            df_drop_change[df_drop_change.Status != "Call(sale)"].index
        )
        if len(df_drop_change_fil) > 0:
            print('send P far')

        if len(df_drop_change_sale) > 0:
            for idx, row in df_drop_change_sale.iterrows():
                add_deal(row)
            print('send MKT team')

#     df_1.to_sql("marketing_zcom", engine, if_exists="replace", index=False)

    if len(df_1) != len(data):
        marketing_zcom()

    return 0

In [2]:
def add_deal(data):
    org_data = {"name": data['Company Name'] or "notset"}
#     response = client.organizations.create_organization(org_data)
#     org_id = response["data"]["id"]

    # create person
    person_data = {
        "name": data['First Name - Last Name'] or "notset",
        "org_id": 'org_id',
        "phone": [data['Phone Number']],
        "email": [data['Email']],
    }
#     response = client.persons.create_person(person_data)
#     person_id = response["data"]["id"]

    # add a deal
    deal_data = {
        "title": data['Company Name'] or "notset",
        "person_id": 'person_id',
        "org_id": 'org_id',
        "stage_id": 12,
    }
    response = client.deals.create_deal(deal_data)
    deal_id = response["data"]["id"]
        
    print(deal_data, deal_id)

In [12]:
# marketing_zcom()

{'title': 'the prodigy group', 'person_id': 'person_id', 'org_id': 'org_id', 'stage_id': 12} 419
send MKT team


0

In [13]:
# deals = client.deals.get_all_deals(params={"limit":"100000"})
# ddata = deals['data']
# ddf = pd.DataFrame(ddata)
# ddf

In [14]:
deal_de = client.deals.get_deal(deal_id=418)

In [15]:
dd = [deal_de['data']]

In [16]:
dadf = pd.DataFrame(dd)

In [17]:
dadf

,id,creator_user_id,user_id,person_id,org_id,stage_id,title,value,currency,add_time,...,owner_name,cc_email,org_hidden,person_hidden,average_time_to_won,average_stage_progress,age,stay_in_pipeline_stages,last_activity,next_activity
0,418,"{'id': 11611129, 'name': 'Sales Tracking', 'em...","{'id': 11611129, 'name': 'Sales Tracking', 'em...","{'active_flag': True, 'name': 'person_id', 'em...","{'name': 'org_id', 'people_count': 0, 'owner_i...",12,the prodigy group,0,THB,2021-02-18 03:13:31,...,Sales Tracking,codium2+deal418@pipedrivemail.com,False,False,"{'y': 0, 'm': 0, 'd': 0, 'h': 0, 'i': 0, 's': ...",0,"{'y': 0, 'm': 0, 'd': 0, 'h': 0, 'i': 36, 's':...","{'times_in_stages': {'12': 2126}, 'order_of_st...",None,None


In [19]:
delete_deal = client.deals.delete_deal(deal_id=419)